In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
992273,2021-02-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
992274,2021-02-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
992275,2021-02-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
992276,2021-02-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27015,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27017,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27019,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27021,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27023,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
615588,2021-02-12,Arkansas,Arkansas,5001,1917,37.00,5000,Arkansas,AR,Arkansas,State,3017804
615590,2021-02-13,Arkansas,Arkansas,5001,1919,37.00,5000,Arkansas,AR,Arkansas,State,3017804
615592,2021-02-14,Arkansas,Arkansas,5001,1926,37.00,5000,Arkansas,AR,Arkansas,State,3017804
615594,2021-02-15,Arkansas,Arkansas,5001,1929,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
992273,2021-02-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
992274,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
992275,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
992276,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-16') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
392,2021-02-16,Snohomish,Washington,30055,519.00,53061,WA,County,822083,3655.96,63.13
782,2021-02-16,Cook,Illinois,466870,9738.00,17031,IL,County,5150233,9065.03,189.08
1171,2021-02-16,Orange,California,257589,3617.00,6059,CA,County,3175692,8111.27,113.90
1559,2021-02-16,Maricopa,Arizona,499865,8505.00,4013,AZ,County,4485414,11144.23,189.61
1947,2021-02-16,Los Angeles,California,1170100,19215.00,6037,CA,County,10039107,11655.42,191.40
...,...,...,...,...,...,...,...,...,...,...,...
991893,2021-02-16,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
992020,2021-02-16,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
992116,2021-02-16,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
992208,2021-02-16,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
392,2021-02-16,Snohomish,Washington,30055,519.00,53061,WA,County,822083,3655.96,63.13,63.13,3655.96
782,2021-02-16,Cook,Illinois,466870,9738.00,17031,IL,County,5150233,9065.03,189.08,189.08,9065.03
1171,2021-02-16,Orange,California,257589,3617.00,6059,CA,County,3175692,8111.27,113.90,113.90,8111.27
1559,2021-02-16,Maricopa,Arizona,499865,8505.00,4013,AZ,County,4485414,11144.23,189.61,189.61,11144.23
1947,2021-02-16,Los Angeles,California,1170100,19215.00,6037,CA,County,10039107,11655.42,191.40,191.40,11655.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
991893,2021-02-16,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
992020,2021-02-16,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
992116,2021-02-16,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
992208,2021-02-16,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
700163,2021-02-16,Clarke,Alabama,3399,46.00,1025,AL,County,23622,14389.13,194.73,194.73,14389.13,1
643543,2021-02-16,Hale,Alabama,2028,59.00,1065,AL,County,14651,13842.06,402.70,402.70,13842.06,2
504081,2021-02-16,Lowndes,Alabama,1294,44.00,1085,AL,County,9726,13304.54,452.40,452.40,13304.54,3
447170,2021-02-16,Etowah,Alabama,12980,298.00,1055,AL,County,102268,12692.14,291.39,291.39,12692.14,4
375486,2021-02-16,Franklin,Alabama,3949,75.00,1059,AL,County,31362,12591.67,239.14,239.14,12591.67,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614621,2021-02-16,Sublette,Wyoming,669,7.00,56035,WY,County,9831,6805.00,71.20,71.20,6805.00,19
944187,2021-02-16,Platte,Wyoming,567,11.00,56031,WY,County,8393,6755.63,131.06,131.06,6755.63,20
848166,2021-02-16,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
762760,2021-02-16,Lincoln,Wyoming,1179,11.00,56023,WY,County,19830,5945.54,55.47,55.47,5945.54,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
504081,2021-02-16,Lowndes,Alabama,1294,44.00,1085,AL,County,9726,13304.54,452.40,452.40,13304.54,3,1
643543,2021-02-16,Hale,Alabama,2028,59.00,1065,AL,County,14651,13842.06,402.70,402.70,13842.06,2,2
445854,2021-02-16,Clay,Alabama,1399,53.00,1027,AL,County,13235,10570.46,400.45,400.45,10570.46,23,3
543438,2021-02-16,Greene,Alabama,854,32.00,1063,AL,County,8111,10528.91,394.53,394.53,10528.91,24,4
215033,2021-02-16,Walker,Alabama,6375,238.00,1127,AL,County,63521,10036.05,374.68,374.68,10036.05,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542129,2021-02-16,Hot Springs,Wyoming,358,3.00,56017,WY,County,4413,8112.40,67.98,67.98,8112.40,16,19
743880,2021-02-16,Uinta,Wyoming,2007,12.00,56041,WY,County,20226,9922.87,59.33,59.33,9922.87,6,20
762760,2021-02-16,Lincoln,Wyoming,1179,11.00,56023,WY,County,19830,5945.54,55.47,55.47,5945.54,22,21
214028,2021-02-16,Teton,Wyoming,3297,9.00,56039,WY,County,23464,14051.31,38.36,38.36,14051.31,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.13,3655.96,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.13,3655.96,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.13,3655.96,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.13,3655.96,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,63.13,3655.96,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989928,2021-02-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
989929,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
989930,2021-02-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
989931,2021-02-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
697497,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14389.13,47,1,2.00
697498,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14389.13,47,1,0.00
697499,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,194.73,14389.13,47,1,1.00
697500,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,194.73,14389.13,47,1,4.00
697501,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,194.73,14389.13,47,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775404,2021-02-12,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
775405,2021-02-13,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
775406,2021-02-14,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
775407,2021-02-15,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
502061,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13304.54,1,3,1.00,0.00
502062,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13304.54,1,3,0.00,0.00
502063,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13304.54,1,3,0.00,0.00
502064,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13304.54,1,3,0.00,0.00
502065,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13304.54,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500416,2021-02-12,Albany,Wyoming,3849,11.00,56001,WY,County,38880,9899.69,28.29,28.29,9945.99,23,5,4.00,0.00
500417,2021-02-13,Albany,Wyoming,3849,11.00,56001,WY,County,38880,9899.69,28.29,28.29,9945.99,23,5,0.00,0.00
500418,2021-02-14,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9945.99,23,5,5.00,0.00
500419,2021-02-15,Albany,Wyoming,3854,11.00,56001,WY,County,38880,9912.55,28.29,28.29,9945.99,23,5,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-16') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
256541,2021-02-16,Imperial,California,26779,583.00,6025,CA,County,181215,14777.47,321.72,321.72,14777.47,1,2,83.00,0.00
1947,2021-02-16,Los Angeles,California,1170100,19215.00,6037,CA,County,10039107,11655.42,191.40,191.40,11655.42,2,5,1196.00,120.00
448438,2021-02-16,Inyo,California,1184,34.00,6027,CA,County,18039,6563.56,188.48,188.48,6563.56,3,31,7.00,0.00
57389,2021-02-16,Stanislaus,California,54484,905.00,6099,CA,County,550660,9894.31,164.35,164.35,9894.31,4,10,159.00,6.00
71106,2021-02-16,Tulare,California,46344,691.00,6107,CA,County,466195,9940.90,148.22,148.22,9940.90,5,9,81.00,0.00
26667,2021-02-16,Riverside,California,286590,3558.00,6065,CA,County,2470546,11600.27,144.02,144.02,11600.27,6,6,1176.00,34.00
406545,2021-02-16,Merced,California,28201,382.00,6047,CA,County,277680,10155.93,137.57,137.57,10155.93,7,8,500.00,7.00
543380,2021-02-16,Kings,California,21658,209.00,6031,CA,County,152940,14161.11,136.65,136.65,14161.11,8,3,11.00,1.00
45695,2021-02-16,San Joaquin,California,65359,1018.00,6077,CA,County,762148,8575.63,133.57,133.57,8575.63,9,18,390.00,26.00
25973,2021-02-16,Fresno,California,93231,1291.00,6019,CA,County,999101,9331.49,129.22,129.22,9331.49,10,14,166.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
943738,2021-02-16,Lassen,California,5546,19.00,6035,CA,County,30573,18140.19,62.15,62.15,18140.19,36,1,48.00,0.00
256541,2021-02-16,Imperial,California,26779,583.00,6025,CA,County,181215,14777.47,321.72,321.72,14777.47,1,2,83.00,0.00
543380,2021-02-16,Kings,California,21658,209.00,6031,CA,County,152940,14161.11,136.65,136.65,14161.11,8,3,11.00,1.00
128029,2021-02-16,San Bernardino,California,282494,2418.00,6071,CA,County,2180085,12957.94,110.91,110.91,12957.94,13,4,0.00,0.00
1947,2021-02-16,Los Angeles,California,1170100,19215.00,6037,CA,County,10039107,11655.42,191.40,191.40,11655.42,2,5,1196.00,120.00
26667,2021-02-16,Riverside,California,286590,3558.00,6065,CA,County,2470546,11600.27,144.02,144.02,11600.27,6,6,1176.00,34.00
160857,2021-02-16,Kern,California,100586,687.00,6029,CA,County,900202,11173.71,76.32,76.32,11173.71,27,7,152.00,0.00
406545,2021-02-16,Merced,California,28201,382.00,6047,CA,County,277680,10155.93,137.57,137.57,10155.93,7,8,500.00,7.00
71106,2021-02-16,Tulare,California,46344,691.00,6107,CA,County,466195,9940.90,148.22,148.22,9940.90,5,9,81.00,0.00
57389,2021-02-16,Stanislaus,California,54484,905.00,6099,CA,County,550660,9894.31,164.35,164.35,9894.31,4,10,159.00,6.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
256541,2021-02-16,Imperial,California,26779,583.00,6025,CA,County,181215,14777.47,321.72,321.72,14777.47,1,2,83.00,0.00
1947,2021-02-16,Los Angeles,California,1170100,19215.00,6037,CA,County,10039107,11655.42,191.40,191.40,11655.42,2,5,1196.00,120.00
448438,2021-02-16,Inyo,California,1184,34.00,6027,CA,County,18039,6563.56,188.48,188.48,6563.56,3,31,7.00,0.00
57389,2021-02-16,Stanislaus,California,54484,905.00,6099,CA,County,550660,9894.31,164.35,164.35,9894.31,4,10,159.00,6.00
71106,2021-02-16,Tulare,California,46344,691.00,6107,CA,County,466195,9940.90,148.22,148.22,9940.90,5,9,81.00,0.00
26667,2021-02-16,Riverside,California,286590,3558.00,6065,CA,County,2470546,11600.27,144.02,144.02,11600.27,6,6,1176.00,34.00
406545,2021-02-16,Merced,California,28201,382.00,6047,CA,County,277680,10155.93,137.57,137.57,10155.93,7,8,500.00,7.00
543380,2021-02-16,Kings,California,21658,209.00,6031,CA,County,152940,14161.11,136.65,136.65,14161.11,8,3,11.00,1.00
45695,2021-02-16,San Joaquin,California,65359,1018.00,6077,CA,County,762148,8575.63,133.57,133.57,8575.63,9,18,390.00,26.00
25973,2021-02-16,Fresno,California,93231,1291.00,6019,CA,County,999101,9331.49,129.22,129.22,9331.49,10,14,166.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14777.47,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3352,02/12/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3353,02/13/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3354,02/14/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00
3355,02/15/21,Lassen,5498,19.00,17983.19,62.15,62.15,18140.19,36,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,321.72,14777.47,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,321.72,14777.47,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3426,02/12/21,Fresno,92168,1291.00,9225.09,129.22,129.22,9331.49,10,14,292.00,8.00
3427,02/13/21,Fresno,92520,1291.00,9260.33,129.22,129.22,9331.49,10,14,352.00,0.00
3428,02/14/21,Fresno,92807,1291.00,9289.05,129.22,129.22,9331.49,10,14,287.00,0.00
3429,02/15/21,Fresno,93065,1291.00,9314.87,129.22,129.22,9331.49,10,14,258.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)